In [ ]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from torch import nn
from torch.nn import functional as F
import torch 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from Sparse.functional import sparse_relu
from torch.nn.functional import relu as original_relu

In [ ]:
dtype = torch.float
device = torch.device("cpu")

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)


In [ ]:
# Create random Tensors for weights.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # To apply our Function, we use Function.apply method. We alias this as 'relu'.
    relu = sparse_relu.apply
    # relu = original_relu

    # Forward pass: compute predicted y using operations; we compute
    # ReLU using our custom autograd operation.
    y_pred = relu(x.mm(w1), torch.tensor(0.05), torch.tensor(1e-6)).mm(w2)
    # y_pred = relu(x.mm(w1)).mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

In [ ]:
lz = (x.mm(w1)<=0)[0].sum()
bz = (x.mm(w1)>0)[0].sum()
print('Number of values closes to 0: {}; bigger: {}'.format(lz, bz))

In [ ]:
from matplotlib import pyplot as plt

a = x.mm(w1)[0].cpu()
plt.hist(a.detach().cpu().numpy(), bins=25)
plt.show()